# 7 WorkFlow

### 7.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer

#### 7.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



*  Si usted eligió modalidad *Gerencial* entonces NO debe modificar nada, su liderazgo logró que otras trabajaron por usted
*  Si usted eligió modalidad de *Analista Junior*, entonces donde dice archivo="gerencial_competencia_2025.csv.gz"  lo debe cambiar por  archivo="analistajr_competencia_2025.csv.gz"
*  Si usted eligió modalidad *Analista Senior*,  ya se las ingenierá SIN preguntar !

In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/labo1r"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


## 7.3  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Tue Jun 24 22:36:11 2025"

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

In [ ]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

#### Parametros
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

In [ ]:
PARAM <- list()
PARAM$semilla_primigenia <- 314159

PARAM$experimento <- 99993
PARAM$dataset <- "analistasr_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [ ]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF", PARAM$experimento,"_algoritmogenetico")
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 7.3.1   Preprocesamiento del dataset

#### 7.3.1.1  DT incorporar dataset

In [2]:
# lectura del dataset
dataset <- fread(paste0("/content/buckets/b1/datasets/", PARAM$dataset))

ERROR: Error in fread(paste0("/content/buckets/b1/datasets/", PARAM$dataset)): could not find function "fread"


#### 7.3.1.2  CA  Catastrophe Analysis
Se intentan reparar las variables que para un mes están con todos los valores en cero.

El método que se utiliza es **Machine Learning** se asigna NA also valores, si ha leido bien, es la "anti imputación de valores faltantes"
<br> Usted podrá aplicar aquí otros métodos

In [3]:
dataset[ foto_mes==202006, internet:=NA]
dataset[ foto_mes==202006, mrentabilidad:=NA]
dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
dataset[ foto_mes==202006, mcomisiones:=NA]
dataset[ foto_mes==202006, mactivos_margen:=NA]
dataset[ foto_mes==202006, mpasivos_margen:=NA]
dataset[ foto_mes==202006, mcuentas_saldo:=NA]
dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

ERROR: Error: object 'dataset' not found


#### 7.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, quizas ajustando por IPC ...
<br>Esta parte podrá ser abordada por todos los Analistas y también la Gerenciapero se decide pedagogicamente no incluirla en esta primer version para reducir la carga cognitiva

In [4]:
# sin codigo en esta primera version del workflow

#### 7.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [5]:
# esta funcion atributos presentes existe debido a que las modalidades poseen datasets con distinta cantidad de campos
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}

# el mes 1,2, ..12
if( atributos_presentes( c("foto_mes") ))
  dataset[, kmes := foto_mes %% 100]

# variable extraida de una tesis de maestria de Irlanda
if( atributos_presentes( c("mpayroll", "cliente_edad") ))
  dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]


ERROR: Error in atributos_presentes(c("foto_mes")): object 'dataset' not found


In [6]:
# visualizo las columas del dataset a esta etapa
colnames(dataset)

ERROR: Error: object 'dataset' not found


#### 7.3.1.4  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest

Esto se mostrará unicamente a la *modalidad Analista Sr*

In [11]:
# No se implementa Feature Engineering a partir de Random Forest

#### 7.3.1.5  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

In [12]:
# Feature Engineering Historico

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


Verificacion de los campos recien creados

In [13]:
ncol(dataset)
colnames(dataset)

[1] 773

[1] "numero_de_cliente"                          
  [2] "foto_mes"                                   
  [3] "active_quarter"                             
  [4] "cliente_vip"                                
  [5] "internet"                                   
  [6] "cliente_edad"                               
  [7] "cliente_antiguedad"                         
  [8] "mrentabilidad"                              
  [9] "mrentabilidad_annual"                       
 [10] "mcomisiones"                                
 [11] "mactivos_margen"                            
 [12] "mpasivos_margen"                            
 [13] "cproductos"                                 
 [14] "tcuentas"                                   
 [15] "ccuenta_corriente"                          
 [16] "mcuenta_corriente_adicional"                
 [17] "mcuenta_corriente"                          
 [18] "ccaja_ahorro"                               
 [19] "mcaja_ahorro"                               
 [20] "mcaja_ahorro_adicional"                     
 [21] "mcaja_ahorro_dolares"                       
 [22] "cdescubierto_preacordado"                   
 [23] "mcuentas_saldo"                             
 [24] "ctarjeta_debito"                            
 [25] "ctarjeta_debito_transacciones"              
 [26] "mautoservicio"                              
 [27] "ctarjeta_visa"                              
 [28] "ctarjeta_visa_transacciones"                
 [29] "mtarjeta_visa_consumo"                      
 [30] "ctarjeta_master"                            
 [31] "ctarjeta_master_transacciones"              
 [32] "mtarjeta_master_consumo"                    
 [33] "cprestamos_personales"                      
 [34] "mprestamos_personales"                      
 [35] "cprestamos_prendarios"                      
 [36] "mprestamos_prendarios"                      
 [37] "cprestamos_hipotecarios"                    
 [38] "mprestamos_hipotecarios"                    
 [39] "cplazo_fijo"                                
 [40] "mplazo_fijo_dolares"                        
 [41] "mplazo_fijo_pesos"                          
 [42] "cinversion1"                                
 [43] "minversion1_pesos"                          
 [44] "minversion1_dolares"                        
 [45] "cinversion2"                                
 [46] "minversion2"                                
 [47] "cseguro_vida"                               
 [48] "cseguro_auto"                               
 [49] "cseguro_vivienda"                           
 [50] "cseguro_accidentes_personales"              
 [51] "ccaja_seguridad"                            
 [52] "cpayroll_trx"                               
 [53] "mpayroll"                                   
 [54] "mpayroll2"                                  
 [55] "cpayroll2_trx"                              
 [56] "ccuenta_debitos_automaticos"                
 [57] "mcuenta_debitos_automaticos"                
 [58] "ctarjeta_visa_debitos_automaticos"          
 [59] "mttarjeta_visa_debitos_automaticos"         
 [60] "ctarjeta_master_debitos_automaticos"        
 [61] "mttarjeta_master_debitos_automaticos"       
 [62] "cpagodeservicios"                           
 [63] "mpagodeservicios"                           
 [64] "cpagomiscuentas"                            
 [65] "mpagomiscuentas"                            
 [66] "ccajeros_propios_descuentos"                
 [67] "mcajeros_propios_descuentos"                
 [68] "ctarjeta_visa_descuentos"                   
 [69] "mtarjeta_visa_descuentos"                   
 [70] "ctarjeta_master_descuentos"                 
 [71] "mtarjeta_master_descuentos"                 
 [72] "ccomisiones_mantenimiento"                  
 [73] "mcomisiones_mantenimiento"                  
 [74] "ccomisiones_otras"                          
 [75] "mcomisiones_otras"                          
 [76] "cforex"                                     
 [77] "cforex_buy"                                

**** ACÁ ARRANCARÍA EL ALGORITMO GENÉTICO ****

#### 7.3.1.6  FEhist Reduccion dimensionalidad con canaritos

Esta etapa solo se mostrará a la *modalidad Anlista Sr* por algun canal secreto de forma de no confundir a los *Analista Jr*  nni distraer con detalles operativos a la estratégica *Modalidad Gerencial*

In [ ]:
# No se implementa la reduccion de la dimensionalidad con canaritos

### 7.3.2 Modelado

#### 7.3.2.1 Training Strategy

Esta etapa de Workflow de  Training Strategy esta pensada para la *Modalidad Gerencial* que posee el dataset de [202005, 202109]
<br> Si usted es un Analista, posee el periodo de [201901, 202109] y deberá experimentar en que meses le conviene experimentar

<br> A la *Modalidad Gerencial* no se le complicada la vida con el undersampling de los continua, por eso PARAM$trainingstrategy$training_pct <- 1.0
<br> Sin embargo, si usted es  *Analista SR* posee un dataset 50 veces ( filas x columnas) más grande que la *Modalidad Gerencial*  y por un tema de velocidad y experimentación más rápida puede llegar a necesitar activar el undersampling de la clase mayoritaria, a pesar de estar corriendo en Google Cloud.

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 202005, 202107 ]  con un undersampling de los CONTINUA del 2% por un tema de velocidad

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling
   * training = [ 202005, 202106 ]  donde se consideran el 100% de los CONTINUA

In [ ]:
PARAM$trainingstrategy$validate <- c(202107)

PARAM$trainingstrategy$training <- c(
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

PARAM$trainingstrategy$training_pct <- 1.0


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [ ]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [ ]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [ ]:
# preparo para que se puede hacer undersampling de los CONTINUA
#  solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]


if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  free_raw_data= TRUE
)

In [ ]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

####  7.3.2.2. Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [ ]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [ ]:
# valor ridiculamente bajo para que corra rapido en el aula y no molestar a la *Modalidad Gerencial*
PARAM$hipeparametertuning$num_interations <- 10

# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  max_bin= 31,
  learning_rate= 0.03,
  feature_fraction= 0.5,
  num_iterations= 2048,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 200
)

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeIntegerParam("num_leaves", lower = 2L, upper = 256L),
  makeIntegerParam("min_data_in_leaf", lower = 2L, upper = 8192L)
)

Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

In [ ]:
# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  message(format(Sys.time(), "%a %b %d %X %Y"))

  # uno la lista de hiperparametros : fijos + variables
  param_completo <- c(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  return(AUC)
}

seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

In [ ]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize= FALSE, # estoy Maximizando AUC
    noisy= FALSE,
    par.set= PARAM$hipeparametertuning$hs,
    has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= "HT.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace = TRUE)
)

Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

In [ ]:
# inicio la optimizacion bayesiana

if (!file.exists("HT.RDATA")) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue("HT.RDATA") # retomo en caso que ya exista
}

la bayesian optimization ha corrido, extraigo los mejores hiperparametros

In [ ]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y, -num_iterations)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file="BO_log.txt",
  sep="\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  list(num_leaves, min_data_in_leaf, num_iterations)
]

print(PARAM$out$lgbm$mejores_hiperparametros)

### 7.3.3 Produccion

### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

#### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [ ]:
PARAM$trainingstrategy$final_train <- c( 202107,
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

# creo el dfinal_train en formato  LightGBM
dfinal_train <- lgb.Dataset(
  data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
  label= dataset[fold_final_train == TRUE, clase01],
  free_raw_data= TRUE
)

nrow( dfinal_train) # verifico el tamaño

#### Final Training Hyperparameters

In [ ]:
# uno los parametros fijos y los mejores encontrados de los variables
fijos <- copy(PARAM$lgbm$param_fijos)

# quito lo que optimice en la Bayesian Optimization
fijos$num_iterations <- NULL
fijos$early_stopping_rounds <- NULL

# agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

#### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [ ]:
final_model <- lgb.train(
  data= dfinal_train,
  param= param_final,
  verbose= -100
)

In [ ]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(final_model, "modelo.txt")

In [ ]:
# ahora imprimo la importancia de variables

tb_importancia <- as.data.table(lgb.importance(final_model))
archivo_importancia <- "impo.txt"

fwrite( tb_importancia,
  file= archivo_importancia,
  sep= "\t"
)

### Scoring

Aplico el modelo final a los datos del futuro

In [ ]:
PARAM$trainingstrategy$future <- c(202109)

dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

In [ ]:
# aplico final_model   a dfuture

prediccion <- predict(
  final_model,
  data.matrix(dfuture[, campos_buenos, with= FALSE])
)

#### Tabla Prediccion

In [ ]:
tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion]

# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

Kaggle Competition Submit

Genero las salidas y hago los submits a Kaggle
<br>El notebook esta preparado para la Modalidad Gerencial, los analistas deben hacer cambios.
<br> Los analistas deben cambiar **competencia** a SU competencia  "labo-i-2025-rosario-analista-jr"   o  la original "labo-i-2025-rosario"  para los Senior
<br> Los cortes  dependen de la cantidad de registros, multiplicar por 2 para los Analistas Jr y por 10 para los Analista Sr

Los Analista Sr luego de meditar cuidadosamente reducirán la cantidad de cortes

In [ ]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

PARAM$kaggle$competencia <- "labo-i-2025-rosario-gerencial"
PARAM$kaggle$cortes <- seq(800, 1300, by = 50)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
  "  semilla=", PARAM$semilla_primigenia,
    "'" )

  linea <- paste( comando, competencia, arch, mensaje)

  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
}

In [ ]:
# grabo los parametros
if( !require("yaml")) install.packages("yaml")
require("yaml")

write_yaml( PARAM, file="PARAM.yml")

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

# Algoritmo genético para Feature Engineer intrames

## Requirements

In [14]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers


Attaching package: ‘ParamHelpers’


The following object is masked from ‘package:R.utils’:

    isVector



Attaching package: ‘mlr’


The following objects are masked from ‘package:R.utils’:

    resample, setThreshold


Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames


The following object is masked from ‘package:R.utils’:

    asInt



Attaching package: ‘smoof’


The following objects are masked from ‘package:R.oo’:

    getDescription, getName




## Selección de variables de entrada

In [15]:
# Defino pool inicial de variables

var_pool <- data.table(
  Feature = cols_lagueables, #Variable
  from1 = NA, #variables 1 usada en iteración anterior
  from2 = NA, #variables 2 usada en iteración anterior
  op = NA, #operación usada para obtenerla
  complete_form = cols_lagueables #forma completa (para uso final)
)

## PARAMS

In [16]:
#hago esto porque en este momento no se como maneja las variables en memoria
meses <- data.frame(dataset$foto_mes)

In [17]:
meses_unique <- as.list(sapply(meses, unique))

In [ ]:
meses_unique[1:26]

[[1]]
[1] 201901

[[2]]
[1] 201902

[[3]]
[1] 201903

[[4]]
[1] 201904

[[5]]
[1] 201905

[[6]]
[1] 201906

[[7]]
[1] 201907

[[8]]
[1] 201908

[[9]]
[1] 201909

[[10]]
[1] 201910

[[11]]
[1] 201911

[[12]]
[1] 201912

[[13]]
[1] 202001

[[14]]
[1] 202002

[[15]]
[1] 202003

[[16]]
[1] 202004

[[17]]
[1] 202005

[[18]]
[1] 202006

[[19]]
[1] 202007

[[20]]
[1] 202008

[[21]]
[1] 202009

[[22]]
[1] 202010

[[23]]
[1] 202011

[[24]]
[1] 202012

[[25]]
[1] 202101

[[26]]
[1] 202102

[[27]]
[1] 202103

[[28]]
[1] 202104

In [ ]:
PARAM$trainingstrategy$validate <- c(202104, 202105)

# PARAM$trainingstrategy$training <- c(
#   202106, 202105, 202104, 202103, 202102, 202101,
#   202012, 202011, 202010, 202009, 202008, 202007,
#   202006, 202005
# )

PARAM$trainingstrategy$training <- c(
    meses_unique[1:26]
)


PARAM$trainingstrategy$training_pct <- 0.1


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [20]:
PARAM$hipeparametertuning$num_interations <- 10

# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  max_bin= 31,
  learning_rate= 0.03,
  feature_fraction= 0.5,
  num_iterations= 2048,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 200
)

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeIntegerParam("num_leaves", lower = 2L, upper = 256L),
  makeIntegerParam("min_data_in_leaf", lower = 2L, upper = 8192L)
)

In [ ]:
PARAM$trainingstrategy$final_train <- c(meses_unique[1:28])

In [22]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

## Funciones

In [23]:
# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  message(format(Sys.time(), "%a %b %d %X %Y"))

  # uno la lista de hiperparametros : fijos + variables
  param_completo <- c(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  return(AUC)
}

get_combinations <- function(cols, n, seed) {
  set.seed(seed)
  combinations <- t(combn(cols, 2))
  if (nrow(combinations) > n) {
    return(combinations[sample(nrow(combinations), n), ])
  } else {
    return(combinations)
  }
}

## Utils

In [24]:
safe_ops <- list(
  `+`    = function(a, b) a + b,
  `-`    = function(a, b) a - b,
  `*`    = function(a, b) a * b,
  `/`    = function(a, b) a / b 
#   pmax   = function(a, b) pmax(a, b, na.rm = TRUE),
#   pmin   = function(a, b) pmin(a, b, na.rm = TRUE)
)

## Cruce + Iteración

In [ ]:
# Numero de generaciones
n_generations <- 3

# Numero de genes
# Tomamos como que siempre se tiene la misma cantidad de variables (genes) y vamos transformandolas.
n_genes <- length(cols_lagueables)

# Numero de mutaciones a realizar
n_mutations <- 40



# Loop principal
for (generation in 1:n_generations) {

    impo_name <- paste0("impo_",generation,".txt")
    seed_loop <- PARAM$semilla_primigenia + generation - 1 #por ahora generamos las seed sumando de a una

    # tener el impo me funciona como marcador, me garantiza que se corrió un ciclo completo
    if (!file.exists(impo_name)) {

        if (generation == 1){
            cols_to_mutate <- var_pool[,Feature]
        } else {
            # importar impo -1 y tomar las primeras 'n_genes'
            var_pool <- fread("var_pool.txt")  #para quedarme con el primero si vengo desde 0
            prev_impo_name <- paste0("impo_",generation-1,".txt") # Si bien puedo reciclar el del loop anterior, si cortó la ejecución no queda en memoria.
            impo <- fread(prev_impo_name)
            rm(prev_impo_name)
            cols_to_mutate <- impo[Feature %in% var_pool[,Feature], Feature][1:n_genes]

            # Si no tengo las variables las creo. En este punto, si retomo el algoritmo debería haber cargado el dataset original, por ende tengo todas
            # las variables originales por las cuales surgen las nuevas variables.

            for (var in cols_to_mutate){
                # Checkeo si existe en el dataset
                if(!(var %in% colnames(dataset)))
                    {
                    dataset <- dataset[, (var) := get(complete_form)]  #creo que esto no anda aun, tengo que traerme la var
                    }
            }


            dataset <- dataset[, (var_pool[,Feature][!var_pool[,Feature] %in% cols_to_mutate]) := NULL] #elimino del dataset las columnas que no voy a usar mas
            rm(impo)

            # deletear cosas que no sirven para no saturar la memoria
            if(exists("dtrain")){rm(dtrain)}
            if(exists("dvalidate")){rm(dvalidate)}
            if(exists("dfinal_train")){rm(dfinal_train)}
        }

        # Elimino columnas creadas para bayesiana y training, si es la primera iteración solo debería dar un warning
        dataset <- dataset[, fold_train := NULL]
        dataset <- dataset[, fold_final_train := NULL]


        ## crear nuevas variables
        # Acá va un algoritmo que God knows cómo será
        combinations <- get_combinations(cols_to_mutate, n_mutations, seed_loop)
        # creo las nuevas variables
        for (var_num in 1:n_mutations){
            var_name <- paste0("gen_",generation,"_n_",var_num)
            var1 <- combinations[var_num, 1]
            var2 <- combinations[var_num, 2]

            # elijo una operación
            inline_seed <- seed_loop*var_num
            set.seed(inline_seed)
            operation_name <- sample(names(safe_ops), 1)
            operation <- safe_ops[[operation_name]]

            # realizo la mutación
            dataset <- dataset[, (var_name) := operation(get(var1),get(var2))]
            complete_form_1 = var_pool[Feature == var1, complete_form]
            complete_form_2 = var_pool[Feature == var2, complete_form]
            new_var <- data.table(
            Feature = var_name,
            from1 = combinations[var_num,1],
            from2 = combinations[var_num,2],
            op = operation_name,
            complete_form = paste0("(",complete_form_1,operation_name,complete_form_2,")")
            )

            # la agregoa al data.table original
            var_pool <- rbind(var_pool, new_var)
        }

        fwrite(var_pool,
           file = "var_pool.txt",
           sep = "\t"
          )

                            ## Bayesiana ###
        # los campos en los que se entrena
        campos_buenos <- copy( setdiff(
            colnames(dataset), c("clase_ternaria","clase01","azar"))
        )

        # preparo para que se puede hacer undersampling de los CONTINUA
        #  solamente por un tema de VELOCIDAD
        set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
        dataset[, azar:=runif(nrow(dataset))]

        # undersampling de los CONTINUA
        dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
            (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
            azar < PARAM$trainingstrategy$training_pct ) ]


        if( !require("lightgbm")) install.packages("lightgbm")
        require("lightgbm")

        dtrain <- lgb.Dataset(
        data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
        label= dataset[fold_train == TRUE, clase01],
        free_raw_data= TRUE
        )

        # datos de validation
        dvalidate <- lgb.Dataset(
        data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
        label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
        free_raw_data= TRUE
        )


        # Esto podría sacarlo pero por ahora lo dejo por el ctrl
        configureMlr(show.learner.output = FALSE)

        # configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
        # por favor, no desesperarse por lo complejo
        obj.fun <- makeSingleObjectiveFunction(
            fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
            minimize= FALSE, # estoy Maximizando AUC
            noisy= FALSE,
            par.set= PARAM$hipeparametertuning$hs,
            has.simple.signature= FALSE # paso los parametros en una lista
        )

        # cada 600 segundos guardo el resultado intermedio
        save_path <- paste0("HT_",generation,".RDATA")
        ctrl <- makeMBOControl(
            save.on.disk.at.time= 600,
            save.file.path= save_path
        )

        # indico la cantidad de iteraciones que va a tener la Bayesian Optimization
        ctrl <- setMBOControlTermination(
            ctrl,
            iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
        )

        # defino el método estandar para la creacion de los puntos iniciales
        #   los "No Inteligentes"
        ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

        # mas configuraciones
        surr.km <- makeLearner(
            "regr.km",
            predict.type= "se",
            covtype= "matern3_2",
            control= list(trace = TRUE)
        )

        # inicio la optimizacion bayesiana

        if (!file.exists(save_path)) {
        bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
        } else {
        bayesiana_salida <- mboContinue(save_path) # retomo en caso que ya exista
        }

        # almaceno los resultados de la Bayesian Optimization
        # y capturo los mejores hiperparametros encontrados

        tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

        # ordeno en forma descendente por AUC = y
        setorder(tb_bayesiana, -y, -num_iterations)

        # grabo para eventualmente poder utilizarlos en OTRA corrida
        file_path <- paste0("BO_",generation,"_log.txt")
        fwrite( tb_bayesiana,
        file=file_path,
        sep="\t"
        )

        # los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
        PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
        1, # el primero es el de mejor AUC
        list(num_leaves, min_data_in_leaf, num_iterations)
        ]

        print(PARAM$out$lgbm$mejores_hiperparametros)

                            ## Final Train ##

        dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

        # creo el dfinal_train en formato  LightGBM
        dfinal_train <- lgb.Dataset(
        data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
        label= dataset[fold_final_train == TRUE, clase01],
        free_raw_data= TRUE
        )

        nrow( dfinal_train) # verifico el tamaño


        # uno los parametros fijos y los mejores encontrados de los variables
        fijos <- copy(PARAM$lgbm$param_fijos)

        # quito lo que optimice en la Bayesian Optimization
        fijos$num_iterations <- NULL
        fijos$early_stopping_rounds <- NULL

        # agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
        param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)
                         
        final_model <- lgb.train(
        data= dfinal_train,
        param= param_final,
        verbose= -100
        )

        modelo_path <- paste0("modelo_",generation,".txt")
        lgb.save(final_model, modelo_path)


        # ahora imprimo la importancia de variables

        tb_importancia <- as.data.table(lgb.importance(final_model))
        archivo_importancia <- impo_name

        fwrite( tb_importancia,
        file= archivo_importancia,
        sep= "\t"
        )
        
    }
}

#### Propuesta Fede

In [ ]:
PARAM$trainingstrategy$future <- c(202106, 202107)  # c(202109)

for (semilla in c(277073, 470779, 314159, 358979, 589793)){
    fijos$seed = semilla

    # agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
    param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)
                     
    # Entrenamiento del modelo
    final_model <- lgb.train(
      data= dfinal_train,
      param= param_final,
      verbose= -100
    )

    # grabo a disco el modelo en un formato para seres humanos ... ponele ...
    lgb.save(final_model, paste0("modelo_", semilla, ".txt"))

    # ahora imprimo la importancia de variables
    tb_importancia <- as.data.table(lgb.importance(final_model))
    archivo_importancia <- paste0("impo_", semilla, ".txt")

    fwrite( tb_importancia,
      file= archivo_importancia,
      sep= "\t"
    )

    #Predicción para datos del futuro
    dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]    

    prediccion <- predict(
      final_model,
      data.matrix(dfuture[, campos_buenos, with= FALSE])
    )

    tb_prediccion <- dfuture[, list(numero_de_cliente, clase_ternaria)]
    tb_prediccion[, prob := prediccion]

    # grabo las probabilidad del modelo
    #  me va a ser util para hacer Ensembles de modelos
    fwrite(tb_prediccion,
      file= paste0("prediccion_", semilla, ".txt"),
      sep= "\t"
    )

    # Elimino cosas de la memoria por las dudas
    rm(final_model)
    rm(tb_importancia)
    rm(dfuture)
    rm(prediccion)
    rm(tb_prediccion)
    
}

#### Cosas Chino

In [26]:
if(!exists("dfinal_train")){

    
}


ERROR: Error: object 'dfinal_train' not found


In [31]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")


In [34]:
m <- lgb.load("modelo_3.txt")

In [49]:
parsed_model <- jsonlite::fromJSON(
    m$dump_model()
)

[1] "numero_de_cliente"                          
  [2] "foto_mes"                                   
  [3] "internet"                                   
  [4] "cliente_edad"                               
  [5] "cliente_antiguedad"                         
  [6] "mrentabilidad"                              
  [7] "mrentabilidad_annual"                       
  [8] "mcomisiones"                                
  [9] "mactivos_margen"                            
 [10] "mpasivos_margen"                            
 [11] "cproductos"                                 
 [12] "tcuentas"                                   
 [13] "mcuenta_corriente"                          
 [14] "ccaja_ahorro"                               
 [15] "mcaja_ahorro"                               
 [16] "mcaja_ahorro_dolares"                       
 [17] "cdescubierto_preacordado"                   
 [18] "mcuentas_saldo"                             
 [19] "ctarjeta_debito"                            
 [20] "ctarjeta_debito_transacciones"              
 [21] "mautoservicio"                              
 [22] "ctarjeta_visa"                              
 [23] "ctarjeta_visa_transacciones"                
 [24] "mtarjeta_visa_consumo"                      
 [25] "ctarjeta_master"                            
 [26] "ctarjeta_master_transacciones"              
 [27] "mtarjeta_master_consumo"                    
 [28] "cprestamos_personales"                      
 [29] "mprestamos_personales"                      
 [30] "mprestamos_prendarios"                      
 [31] "mprestamos_hipotecarios"                    
 [32] "cplazo_fijo"                                
 [33] "mplazo_fijo_dolares"                        
 [34] "minversion1_pesos"                          
 [35] "minversion2"                                
 [36] "ccaja_seguridad"                            
 [37] "cpayroll_trx"                               
 [38] "mpayroll"                                   
 [39] "ccuenta_debitos_automaticos"                
 [40] "mcuenta_debitos_automaticos"                
 [41] "ctarjeta_visa_debitos_automaticos"          
 [42] "mttarjeta_visa_debitos_automaticos"         
 [43] "mpagomiscuentas"                            
 [44] "ccomisiones_mantenimiento"                  
 [45] "mcomisiones_mantenimiento"                  
 [46] "ccomisiones_otras"                          
 [47] "mcomisiones_otras"                          
 [48] "ctransferencias_recibidas"                  
 [49] "mtransferencias_recibidas"                  
 [50] "ctransferencias_emitidas"                   
 [51] "mtransferencias_emitidas"                   
 [52] "mextraccion_autoservicio"                   
 [53] "ccheques_emitidos"                          
 [54] "mcheques_emitidos_rechazados"               
 [55] "tcallcenter"                                
 [56] "ccallcenter_transacciones"                  
 [57] "thomebanking"                               
 [58] "chomebanking_transacciones"                 
 [59] "ccajas_transacciones"                       
 [60] "ccajas_consultas"                           
 [61] "ctrx_quarter"                               
 [62] "tmobile_app"                                
 [63] "cmobile_app_trx"                            
 [64] "Master_delinquency"                         
 [65] "Master_status"                              
 [66] "Master_mfinanciacion_limite"                
 [67] "Master_Fvencimiento"                        
 [68] "Master_Finiciomora"                         
 [69] "Master_msaldototal"                         
 [70] "Master_msaldopesos"                         
 [71] "Master_mconsumospesos"                      
 [72] "Master_mlimitecompra"                       
 [73] "Master_fultimo_cierre"                      
 [74] "Master_mpagospesos"                         
 [75] "Master_fechaalta"                           
 [76] "Master_mpagominimo"                         
 [77] "Visa_delinquency"                          

In [50]:
names <- parsed_model$feature_names

In [53]:
var_pool <- fread("var_pool.txt")

In [70]:
complete_form <- var_pool[var_pool$Feature == "gen_3_n_11",][[5]]

In [73]:
complete_form

[1] "((thomebanking/Visa_mpagominimo)-ccheques_emitidos)"

In [75]:
for (var in names){
                # Checkeo si existe en el dataset
                if(!(var %in% colnames(dataset)))
                    {
                    complete_form <- var_pool[var_pool$Feature == var,][[5]] 
                    dataset <- dataset[, (var) := eval(parse(text = complete_form))]
                    }
            }

Warning message in `[.data.table`(dataset, , `:=`((var), eval(parse(text = complete_form)))):
“Tried to assign NULL to column 'fold_train', but this column does not exist to remove”
Warning message in `[.data.table`(dataset, , `:=`((var), eval(parse(text = complete_form)))):
“Tried to assign NULL to column 'fold_final_train', but this column does not exist to remove”
